In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install  faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 53.8 MB/s eta 0:00:00


In [3]:
!pip install accelerate==0.29.3 bitsandbytes==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [4]:
import tensorflow as tf
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig)

In [5]:
from google.colab import userdata
HF_TOKEN=userdata.get('huggingface')

In [6]:
model_name = "nomic-ai/nomic-embed-text-v1.5"

In [19]:
df= pd.read_csv("/content/song_data.csv")

In [20]:
song_df =df.add_prefix("song_")

In [21]:
song_df = song_df.drop(columns=["song_id", "song_release_date"])

In [22]:
def fix_artists(str_list):
    return ", ".join([v for v in str_list.rstrip("']").lstrip("['").split("', '")])


song_df["song_artists"] = song_df["song_artists"].apply(fix_artists)

In [23]:
song_df.insert(
    0, "song_description", song_df["song_name"] + " - " + song_df["song_artists"]
)

In [24]:
song_data = song_df[
    ~song_df.duplicated(subset=["song_description"], keep="first")
].reset_index(drop=True)

In [25]:
song_df.shape

(170653, 18)

In [26]:
song_df= song_df.iloc[165653:170653]


In [27]:
song_df.shape

(5000, 18)

In [28]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [29]:
import transformers

In [30]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00


In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

model = transformers.AutoModel.from_pretrained(model_name,
    device_map="cuda",
    quantization_config=bnb_config,
    token=HF_TOKEN)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for nomic-ai/nomic-embed-text-v1.5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/nomic-ai/nomic-embed-text-v1.5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling_hf_nomic_bert.py:   0%|          | 0.00/84.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

In [32]:
def get_embedding(text, idx):
    inputs= tokenizer(text, return_tensors="pt")
    outputs= model(**inputs)
    print (idx)
    return outputs.last_hidden_state[:,0, :].detach().numpy()

# When calling the function:
song_df['embedding'] = song_df.apply(lambda row: get_embedding(row['song_description'], row.name), axis=1)
                                                          # Use row.name to get index of the row

165653
165654
165655
165656
165657
165658
165659
165660
165661
165662
165663
165664
165665
165666
165667
165668
165669
165670
165671
165672
165673
165674
165675
165676
165677
165678
165679
165680
165681
165682
165683
165684
165685
165686
165687
165688
165689
165690
165691
165692
165693
165694
165695
165696
165697
165698
165699
165700
165701
165702
165703
165704
165705
165706
165707
165708
165709
165710
165711
165712
165713
165714
165715
165716
165717
165718
165719
165720
165721
165722
165723
165724
165725
165726
165727
165728
165729
165730
165731
165732
165733
165734
165735
165736
165737
165738
165739
165740
165741
165742
165743
165744
165745
165746
165747
165748
165749
165750
165751
165752
165753
165754
165755
165756
165757
165758
165759
165760
165761
165762
165763
165764
165765
165766
165767
165768
165769
165770
165771
165772
165773
165774
165775
165776
165777
165778
165779
165780
165781
165782
165783
165784
165785
165786
165787
165788
165789
165790
165791
165792
165793
165794
165795

In [63]:
song_df.tail()

,song_description,song_name,song_artists,song_acousticness,song_danceability,song_duration_ms,song_energy,song_explicit,song_instrumentalness,song_key,song_liveness,song_loudness,song_mode,song_popularity,song_speechiness,song_tempo,song_valence,song_year,embedding
170648,"China - Anuel AA, Daddy Yankee, KAROL G, Ozuna...",China,"Anuel AA, Daddy Yankee, KAROL G, Ozuna, J Balvin",0.08460,0.786,301714,0.808,0,0.000289,7,0.0822,-3.702,1,72,0.0881,105.029,0.608,2020,"[[-0.9181382, 1.0350647, -4.9642863, 0.3046093..."
170649,Halloweenie III: Seven Days - Ashnikko,Halloweenie III: Seven Days,Ashnikko,0.20600,0.717,150654,0.753,0,0.000000,7,0.1010,-6.020,1,68,0.0605,137.936,0.734,2020,"[[-1.4549726, -0.36402673, -4.7480397, 0.86181..."
170650,AYA - MAMAMOO,AYA,MAMAMOO,0.10100,0.634,211280,0.858,0,0.000009,4,0.2580,-2.226,0,76,0.0809,91.688,0.637,2020,"[[-0.018719627, 0.7859725, -4.6667376, -0.8826..."
170651,Darkness - Eminem,Darkness,Eminem,0.00998,0.671,337147,0.623,1,0.000008,2,0.6430,-7.161,1,70,0.3080,75.055,0.195,2020,"[[-0.6409477, -1.0746467, -4.0126133, 0.418130..."
170652,"Billetes Azules (with J Balvin) - KEVVO, J Balvin",Billetes Azules (with J Balvin),"KEVVO, J Balvin",0.13200,0.856,189507,0.721,1,0.004710,7,0.1820,-4.928,1,74,0.1080,94.991,0.642,2020,"[[-0.18899862, -0.24449186, -5.53311, -0.21188..."


[[-1.33515072e+00 -5.41150987e-01 -5.33342505e+00  7.83291161e-01
  -4.13441986e-01  5.26535213e-01 -2.99686462e-01  5.15111208e-01
  -1.17726617e-01 -2.85022944e-01 -3.21794271e-01  2.25594919e-02
  -6.35669410e-01 -1.91857040e-01  1.00511769e-02 -1.84657443e+00
  -4.20984067e-02  2.75942236e-02 -6.84849739e-01 -5.99823892e-01
  -5.34127891e-01  4.18277271e-02 -8.14297974e-01 -6.76193118e-01
   1.86043406e+00  5.06093681e-01  6.28507972e-01  7.86414802e-01
   8.31965685e-01  2.04771185e+00  2.84479213e+00 -1.56455243e+00
   1.18086800e-01 -1.13006055e+00 -2.02541852e+00 -1.19301867e+00
  -6.07633173e-01  9.30519402e-02 -1.36782065e-01  1.72794843e+00
   1.05820346e+00  1.16962337e+00  1.59616423e+00  1.46987364e-01
   1.14878404e+00 -1.24762368e+00  1.86595678e-01  5.13901591e-01
  -7.41939843e-01 -7.09044337e-02  1.54556513e+00  3.55912596e-01
  -2.04593849e+00 -6.61794543e-01  9.07009780e-01 -7.65479922e-01
   5.36339343e-01  1.07632136e+00 -9.83702183e-01  7.33994961e-01
   2.46534

In [35]:
from sklearn.preprocessing import StandardScaler

In [47]:
numeric_cols = list(
    song_df.drop(columns=["song_name", "song_artists", "song_description","embedding"]).columns
)
numeric_cols

['song_acousticness',
 'song_danceability',
 'song_duration_ms',
 'song_energy',
 'song_explicit',
 'song_instrumentalness',
 'song_key',
 'song_liveness',
 'song_loudness',
 'song_mode',
 'song_popularity',
 'song_speechiness',
 'song_tempo',
 'song_valence',
 'song_year']

In [44]:
song_df.shape

(5000, 19)

In [48]:

scaler = StandardScaler()
norm_num_cols=scaler.fit_transform(song_df[numeric_cols])

In [50]:
song_df['embedding'].values.shape

(5000,)

In [49]:
comb_f= np.hstack([np.vstack(song_df['embedding'].values),norm_num_cols])

In [51]:
comb_f.shape

(5000, 783)

In [52]:
comb_f=comb_f.astype('float32')

In [54]:
import faiss
faiss.normalize_L2(comb_f)

In [57]:
d=comb_f.shape[1]

In [58]:
index= faiss.IndexFlatL2(d)
index.add(comb_f)

In [59]:
faiss.write_index(index, 'vector_index.faiss')

In [61]:
default_numeric_value= np.zeros(norm_num_cols.shape[1])

In [77]:
def get_embeddings(text):
    inputs= tokenizer(text, return_tensors="pt")
    outputs= model(**inputs)

    return outputs.last_hidden_state[:,0, :].detach().numpy()








def get_query_embedding(query_text, default_numeric_value):
    query_embedding = get_embeddings(query_text)
    # Reshape default_numeric_value to have 2 dimensions
    default_numeric_value = default_numeric_value.reshape(query_embedding.shape[0], -1)
    query_combined = np.hstack([query_embedding, default_numeric_value])
    # Ensure query_combined is a float32 NumPy array
    query_combined = query_combined.astype(np.float32) # Change the data type to float32
    faiss.normalize_L2(query_combined)
    return query_combined

def recommend(query_text, num_recommendations=5):
    query_embedding = get_query_embedding(query_text, default_numeric_value) # And here
    D, I = index.search(query_embedding.reshape(1, -1), num_recommendations)
    recommendation= song_df.iloc[I[0]][['song_name',	'song_artists','song_year']].copy()
    recommendation['similarity_score'] = 1-D[0]
    return recommendation

In [81]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [82]:
# Example usage
query_text = "Darkness - Eminem"
recommendations = recommend(query_text)
print(recommendations)

                        song_name                                     song_artists  song_year  similarity_score
170651                   Darkness                                           Eminem       2020          0.951195
170376                In The Dark                                               YG       2019          0.507429
168505        To Darkness / Kripa  Dharohar Project, Laura Marling, Mumford & Sons       2010          0.493471
169643           The Blackest Day                                     Lana Del Rey       2015          0.460582
166704  Blacking Out the Friction                              Death Cab for Cutie       2001          0.422949
